In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import torchvision 
from tqdm import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import soundfile as sf

device = 'cpu'
if torch.cuda.is_available():
  device = 'cuda'
  torch.cuda.manual_seed_all(1)

In [2]:
!mkdir sp_covid
!mkdir sp_covid/TRAIN
!mkdir sp_covid/TRAIN/covid
!mkdir sp_covid/TRAIN/no_covid
!mkdir sp_covid/TRAIN/asymp
!mkdir sp_covid/TEST
!mkdir sp_covid/TEST/covid
!mkdir sp_covid/TEST/no_covid
!mkdir sp_covid/TEST/asymp

In [12]:
def wave_to_specgram(folder_path):    
    file_list = os.listdir(folder_path)
    dataset = torch.empty(1)
    for file in file_list:
        if file != '.DS_Store':
            with sf.SoundFile(folder_path+'/'+file) as f:
                duration = (len(f) / f.samplerate)
            if (duration < 0.5):
                os.remove(folder_path+'/'+file)
            else:
                waveform, sample_rate = torchaudio.load(folder_path+'/'+file)
                specgram = torchaudio.transforms.Spectrogram()(waveform)
                specgram_resize = torchvision.transforms.Resize((300,300))(specgram)
                plt.figure(frameon=False)
                plt.axis('off')
                #print(list(specgram_resize.shape[1:]))
                #We add epsilon = 1e-12 in order to be able to compute the logarithm and not get NaN values in the images
                specgram_resize += torch.ones(list(specgram_resize.shape))*1e-12
                plt.imshow(specgram_resize.log2()[0,:,:].numpy(), cmap='gray')
                plt.savefig('./sp_covid/'+folder_path.split('/')[-2]+'/'+folder_path.split('/')[-1]+'/'+file.strip('.wav')+'.png',bbox_inches = 'tight',pad_inches = 0)
                plt.close()

In [ ]:
train_covid_dir = './Cough_dataset/Labeled_audio/cough/TRAIN/covid'
train_nocovid_dir = './Cough_dataset/Labeled_audio/cough/TRAIN/no_covid'
train_asymp_dir = './Cough_dataset/Labeled_audio/cough/TRAIN/asymp'
test_covid_dir = './Cough_dataset/Labeled_audio/cough/TEST/covid'
test_nocovid_dir = './Cough_dataset/Labeled_audio/cough/TEST/no_covid'
test_asymp_dir = './Cough_dataset/Labeled_audio/cough/TEST/asymp'


wave_to_specgram(train_covid_dir)
wave_to_specgram(train_nocovid_dir)
wave_to_specgram(train_asymp_dir)
wave_to_specgram(test_covid_dir)
wave_to_specgram(test_nocovid_dir)
wave_to_specgram(test_asymp_dir)

./Cough_dataset/Labeled_audio/cough/TRAIN/covid b7mMUQm5bObj1jwGFaNyaR07pt83_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid ZaeW7vuikQgf7984JVqXYOtUQMs2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid 2vH4QMrMaMMm5oZYScY0b36Z8MC3_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid gpcoorcyfuXQZxcUFTxthUKOua03_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid ffyNrzu5ZGXrZ0I7MbxBwqbtIln1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid xVC8kMuWNme6VSFOvNVT0j21Qw12_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid BTq6ZzkM2pdcUmgtOHVP4GG5wvZ2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid hVZQcrz6cfaOMfL3jpRqJBUiumF2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid iS29Ewab4XQHypUcZLygL7AYH6J2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid 8fdUOI5UtkZBCVFXNT1a4hl3rX33_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid 0KSi2atlmsXN

./Cough_dataset/Labeled_audio/cough/TRAIN/covid y0TkG5sfyhadW3VSct50RTl3HN02_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid DGWrVMD5yZc5jcjMzblC4kaBLmK2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid IFMid2LmeCTSQAFaL8wTSUg20No2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid wXChdMlWemXwRdX642WnX6JbYfA2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid hLVaY0Dox1bLCTAqoq0wKKOqBjc2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid fWUMPPgD78VZinzLLLwqbyTysJa2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid b7mMUQm5bObj1jwGFaNyaR07pt83_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid 85iNcV2z6MNT3FrygbEC4fuGrIu1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid y0TkG5sfyhadW3VSct50RTl3HN02_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid 4qLi2eWkYGasKhmmPPiyFAGUqBr2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/covid ZzKpUtFGzsVEK7E1

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid QC2olizheGbckya2rPjOWhn00vp1_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 14bqYTAafsSLdCtHZVw1gavRhpd2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 80M2CERHxJcqnfMAjwyheSC67Ec2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 6mCAHbyEebbEDkxHxOqihoJRkX83_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid g9WjuKnjoYZYmLVDY86u4JIwZ333_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid N49IPlluNddotKfNlRNVJJ2Ad6E3_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 3CMQ3nCRdaU5K8pfUiqOpzUgQRz2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid YKJ4AmuaSThm9fM7nAE2rxLs9Bm2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 97Rb1xwK1GTHQL9OqkuoKV18Zug2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid Fohk6SJxcJVo4rtuGD6PMwJ9qTQ2_cough-heavy.wav
./Cough_dataset/Labeled_audi

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 96vge8k4YhZKRuYBP7T1WVw2Ouz2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid rf6smT3AxxYh5E5LqwXPy0VHrRr1_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 3VWhvtBFHKMgEKxlXzMYeg5NeBN2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid dmE3r0LE6Yg9qmCd7aubpgug9te2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid g0gdNMhQIbUlZRTod20TdtqafgJ3_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid xb5Zffvkf5MqrUPju4A98njfRMy1_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid XVjp63ocWZRqJRdxCtklUfmdklm2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid aohEEUYh09Nbb1k0HlGwxPcuD663_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid oTCyHYFIjtezHe2Tr9yZg8Lj5sv2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 6C5Cxv34nzSpBc7hvwkP7p5olLz1_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid Uy5OhcHEQXY65CsYbiRPtN7RVBf1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 1ctsxp37dWNbCbizTioZzpAWqJx2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 1j2LqFPOJIQ8nTlovQqNr6bdUMl1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid t19ZtqDNiJd7nL1SntNpS8KQDO73_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid YR33hHou5PgrLYv2L1mLzHE4na12_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid IQ7tfWcSsRXGe0J94ouY9rsTB8V2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid A1tmg4RT7Mekg8LKxGC2agpfmdi1_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid k7R0sHWreMTQCDnV5dMzd7t101A3_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid VQVtmKC3HwVHbY1xMTued5OOP4E2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid WyCgdIfGzxXRPeimIoqiPVysAHB2_cough-heavy.wav
./Cough_dataset/Labeled_audio/co

...

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid HhIoOCxCe1Z02gC8qa7RsZZ8Vz23_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid n8WR98Nf2WMncTf0mEVL8lPCnp12_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid f4tk6UjR0Ldbnq5tRfgYhT6VGqV2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 5JO89F9hCpVbVFy7jFqPKtVRyIV2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid Y5eS0NFkNCPdRy7EFiTvm5QJ4qO2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid giXzRAAuoRRAg7xQ9Y56DX0FVDr1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid NakKwA95snbM0W6Lk1nV0ptOmOA2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 5w8ZIMTjkXOMI78Q3sccWcvMKiI2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid M1RgcbGDbWbsxNkmPHlrh9J9n572_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid oNVwfrIb63g5zB4jsaduCjuufPo1_cough-heavy.wav
./Cough_dataset/Labeled_audi

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid CDJLmx0CWdNRiAvgFwn2aHonth43_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid B6X0O4W1xyP5lMLvMIzc4fGhQSj1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 9qXBPCugOSe0Yl1GhThi1kCVZDD2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 3fKAFFPrGTZTay5GlkoipIjRUPL2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid AxuYWBN0jFVLINCBqIW5aZmGCdu1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid YiFpl4utCGTCGxQHqjYMOUgKlV63_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid V0Wtp76p8QaDtvEnAsgSimyMspH3_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid LyyyoHyVVygA9sFTiv5SWaaUR2E3_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid cog0Kv05iPM162wFPDfBEnEqpzJ3_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid CEn02KlCBsUgUqUmf5BfUgaITG33_cough-heavy.wav
./Cough_dataset/Labeled_audio/

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid byVsWVHvkgc9Pkf4lUPOGSdZ0Vl2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 11GcRROEvCReCSBzteGoFLw1GyS2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid xxeoHryZgjVJhQiY6LAR7EDABMn2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 22oBvWaWwBhbdNfWoshbse1qZ4q2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 8J17ZUDTdXfN5NYwjjp1ANcRqbJ2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid yCzbULubsuY5sNYwje0zqNqtEB62_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid cAdvtOxMKHg9hO3IbimCRK9Vooi1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid FGRDO4IBbAejR0WHD5YbkXTCasg2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid QOmSmWWvcLOxZgzrV4u04ByoebE2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid EPA8c37Cb5e2aaUfisoVCzoxMnT2_cough-shallow.wav
./Cough_dataset/Labeled_

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid tAqfUCGTWKdgIF95SfZU0gtLPZt1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid ByvAxiKJC5QyNZi2YJu6GyZYgPs1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid i0reS77d27TfHnKANrEGHVZXkLw2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid av4p9I9aBGfNYwrwkh4jEiEk1lE2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid Ms3fiiaGeta2PNzjXEE3iMBmh1h1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid IP6mdjKfmKNgsLLT110MkgGvWK02_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 6GBpxEuVACapBiyIgwQOAj1zuuy1_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid tmguPZG0j4X5PktnGJyOYxVGtqW2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 3RBbwztykUWGLJ5UmBZoL5pqOtA3_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid eD46zs0Gr9Rm2dRnRvkdjI0jc953_cough-shallow.wav
./Cough_dataset/Labeled_au

./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid MEEGIoNSX4exg4oIL3Q9MdysPJq1_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid k9c8ik6XlhbYFnHoXHqyDIQ5axe2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid sQOS8ZFe4wN6uXSrjtI5ivCt92C3_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 8PtpwfiaGpMFqciQAnNjDnnbvjw2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 6WQPyHp9AZThZUTD1Z2rOXv3d5R2_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid KoVhPh3wszMRh4pyxUzNhhSwxyX2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid pBB2aZFbLCMky5VPRAoWIClKN2Y2_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid Z0JZ3bVzYZcEx2GkbTKAyWbyC2J3_cough-heavy.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid 1iqECx2FTmg6fkTzjVIZeN7ymnH3_cough-shallow.wav
./Cough_dataset/Labeled_audio/cough/TRAIN/no_covid goCuK84Tklfu22gFPPMqt1ZIjxo2_cough-shallow.wav
./Cough_dataset/Labeled_audio/